In [ ]:
%pip install qiskit numpy pylatexenc

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator

import matplotlib.pyplot as plt

import numpy as np

We will implement ''Grover Adaptive Search for Constrained Polynomial Binary Optimization''. We will write a Qiskit function that takes three inputs:
*   a pseudo-boolean function $f: \mathbb{F}_2^n\to \mathbb{Z}$
*   a threshold, $t\in \mathbb{Z}$
*   a constraint function $C: \mathbb{F}_2^n\to \mathbb{Z}$

and outputs marker oracle $U_{f,t,C}$ such that
$$U_{f,t,C}|x⟩_n|y⟩_1=\begin{cases}|x⟩_n|y\oplus 1⟩_1, \; \mathrm{if}\; f(x)>t \; \mathrm{ and } \; C(x)\ge 0 \\|x⟩_n|y⟩_1, \; \mathrm{otherwise.}
\end{cases}$$

We will implement GAS by the following steps:
*   Realize function $f$ and $C$ by circuits (For simplicity, assume $f,C$ are polynomials)
*   Implement the oracle $U_{f,t,C}$, seting the constraints to be $f>t \,\wedge\,C\ge 0$.
*   Apply Grover and measure to find a candidate $x^{(0)}$.
*   Recursively run Grover with updated $t=f(x^{(0)})$.


### Step 1: Realize functions

In [ ]:
# ===========================================
# Pseudo-Boolean evaluator (Qiskit)
# ===========================================
# - Term representation for F(x) with signed weights
# - Bit-width + offset sizing for signed range
# - Ancilla-free QFT constant adder (unchanged)
# - Reversible evaluator:
#     |x>|0...0> -> |x>| F(x) + offset  (mod 2^m) >
#
# Conventions:
# - Accumulator is LITTLE-ENDIAN: acc[0] = LSB, acc[m-1] = MSB
# - Addition is modulo 2^m
# ===========================================

from math import ceil, log2
from typing import List, Tuple, Callable
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister

Term = Tuple[int, List[int]]  # (weight (can be negative), [indices of x in the monomial])


# -------------------------------
# Bit-width & offset (from terms)
# -------------------------------
def signed_bits_and_offset_from_terms(terms: List[Term]) -> tuple[int, int]:
    """
    Compute minimal unsigned bit-width m and an offset >=0 so that for all x:
        F'(x) := F(x) + offset ∈ [0, 2^m - 1].

    We bound F(x) by summing positive and negative weights separately:
        F_min = sum_{w<0} w,  F_max = sum_{w>0} w.
    Then choose:
        offset = -F_min,  width = F_max - F_min,  m = ceil(log2(width+1)) (at least 1).

    Returns:
        (m, offset)
    """
    pos = sum(w for w, _ in terms if w > 0)
    neg = sum(w for w, _ in terms if w < 0)  # <= 0
    width = pos - neg  # = F_max - F_min >= 0
    m = max(1, ceil(log2(width + 1)))
    offset = -neg
    return m, offset


# -------------------------------
# Ancilla-free QFT constant adder
# -------------------------------
# Implements |v> -> |(v + k) mod 2^m> on an m-qubit LITTLE-ENDIAN register.

def _qft(n: int) -> QuantumCircuit:
    """Non-swapping QFT on n qubits (little-endian: qubit 0 is LSB)."""
    qr = QuantumRegister(n, "a")
    qc = QuantumCircuit(qr, name=f"QFT_{n}")
    for j in range(n):
        qc.h(qr[j])
        for k in range(j + 1, n):
            qc.cp(np.pi / (1 << (k - j)), qr[k], qr[j])
    return qc

def quantum_adder(k: int, n: int) -> QuantumCircuit:
    """
    Unsigned constant adder on an n-qubit LITTLE-ENDIAN register:
        |v> -> |(v + k) mod 2^n>
    """
    k_mod = k % (1 << n)
    qr = QuantumRegister(n, "acc")
    qc = QuantumCircuit(qr, name=f"+{k_mod}")
    qc.compose(_qft(n), qr, inplace=True)
    for j in range(n):
        angle = 2 * np.pi * k_mod / (1 << (j + 1))
        if angle != 0.0:
            qc.p(angle, qr[j])
    qc.compose(_qft(n).inverse(), qr, inplace=True)
    return qc


# -------------------------------
# Reversible signed evaluator
# -------------------------------
def build_eval_from_polynomial(
    n_vars: int,
    terms: List[Term],
    acc_bits: int | None = None,
    offset: int | None = None,
    adder_fn: Callable[[int, int], QuantumCircuit] = quantum_adder,
    label: str = "E_f_signed",
) -> QuantumCircuit:
    """
    Build E_f (signed weights allowed):
        |x>|0...0>_acc  ->  |x>| ( F(x) + offset ) mod 2^m >_acc

    where F(x) = sum_{(w,S)} w * Π_{i∈S} x_i with w ∈ ℤ.
    The (m, offset) are chosen so F(x)+offset is always nonnegative.

    Args:
        n_vars   : number of x-qubits (variables).
        terms    : list of (w, S) with w ∈ ℤ and S ⊆ {0..n_vars-1}.
        acc_bits : accumulator width m; if None, auto-sized from terms.
        offset   : unsigned offset; if None, auto-chosen from terms.
        adder_fn : function returning a constant-adder (+k) on m qubits.
        label    : name for the returned circuit.

    Returns:
        QuantumCircuit over registers [x(n_vars), acc(acc_bits)].

    Notes:
        - We add the offset first, then for each term add k = (w mod 2^m) controlled on S.
          Negative w are implemented as modular additions: (2^m - |w|).
        - Ensure acc_bits is large enough (we auto-compute if not provided).
    """
    # Validate indices
    if any(any(i < 0 or i >= n_vars for i in S) for (_, S) in terms):
        raise ValueError("Some term references an x index out of range.")

    # Auto-size accumulator and offset if not provided
    if acc_bits is None or offset is None:
        m_auto, off_auto = signed_bits_and_offset_from_terms(terms)
        acc_bits = m_auto if acc_bits is None else acc_bits
        offset = off_auto if offset is None else offset
    if acc_bits < 1:
        raise ValueError("acc_bits must be >= 1.")
    if offset < 0:
        raise ValueError("offset must be nonnegative.")

    M = 1 << acc_bits

    # Build circuit
    x = QuantumRegister(n_vars, "x")
    acc = QuantumRegister(acc_bits, "acc")
    qc = QuantumCircuit(x, acc, name=label)

    # 0) Add the offset unconditionally
    if (offset % M) != 0:
        qc.append(adder_fn(offset % M, acc_bits).to_gate(label=f"+{offset%M}"), qargs=[*acc])

    # 1) Add each term modulo 2^m (handles negative weights via Python %)
    for w, S in terms:
        if w == 0:
            continue
        k = w % M
        add_gate = adder_fn(int(k), acc_bits).to_gate(label=f"+{k}")
        if len(S) == 0:
            qc.append(add_gate, qargs=[*acc])
        elif len(S) == 1:
            qc.append(add_gate.control(1), qargs=[x[S[0]], *acc])
        else:
            qc.append(add_gate.control(len(S)), qargs=[*(x[i] for i in S), *acc])

    return qc

## Step 2: Implement the oracle





In [ ]:
from typing import Callable  # ensure available
from qiskit.circuit.library import IntegerComparator, MCXGate
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Gate

def build_oracle_U_f_t_C(
    n_vars: int,
    terms_f: list[tuple[int, list[int]]],
    t: int,
    terms_c: list[tuple[int, list[int]]],
    c_threshold: int = 0,
    f_bits: int | None = None,
    c_bits: int | None = None,
    adder_fn: Callable[[int, int], QuantumCircuit] = quantum_adder,
    label: str = "U_f>t_and_C>=cthr",
) -> Gate:
    # ----- sizing & offsets for SIGNED -----
    mf_auto, off_f = signed_bits_and_offset_from_terms(terms_f)
    mc_auto, off_c = signed_bits_and_offset_from_terms(terms_c) if terms_c else (1, 0)

    if f_bits is None:
        f_bits = mf_auto
    if c_bits is None:
        c_bits = mc_auto

    # sanity: user-specified width must fit conservative range
    if f_bits < mf_auto:
        raise ValueError(f"f_bits={f_bits} too small for signed F range (need >= {mf_auto}).")
    if c_bits < mc_auto:
        raise ValueError(f"c_bits={c_bits} too small for signed C range (need >= {mc_auto}).")

    # ----- interface registers -----
    x = QuantumRegister(n_vars, "x")
    y = QuantumRegister(1, "y")
    f_acc = QuantumRegister(f_bits, "f")
    f_flag = QuantumRegister(1, "ff")
    c_acc = QuantumRegister(c_bits, "c")
    c_flag = QuantumRegister(1, "cf")

    U = QuantumCircuit(x, y, f_acc, f_flag, c_acc, c_flag, name=label)

    # ----- 1) compute F'(x) = F(x) + off_f (signed eval) -----
    E_f = build_eval_from_polynomial(
        n_vars=n_vars, terms=terms_f, acc_bits=f_bits, offset=off_f,
        adder_fn=adder_fn, label="E_f_signed"
    )
    U.compose(E_f, qubits=[*x, *f_acc], inplace=True)

    # ----- 2) flag F > t  <=>  F' >= (t + off_f + 1) -----
    comp_f, af, f_set_const_true = None, None, False
    t_prime = t + off_f + 1
    if t_prime <= 0:
        U.x(f_flag[0])           # always true
        f_set_const_true = True
    elif t_prime >= (1 << f_bits):
        pass                      # impossible: ff stays 0
    else:
        comp_f = IntegerComparator(num_state_qubits=f_bits, value=int(t_prime), geq=True)
        af_n = comp_f.num_qubits - (f_bits + 1)
        af = QuantumRegister(af_n, "af") if af_n > 0 else None
        if af is not None:
            U.add_register(af)
            U.compose(comp_f, qubits=[*f_acc, f_flag[0], *af], inplace=True)
        else:
            U.compose(comp_f, qubits=[*f_acc, f_flag[0]], inplace=True)

    # ----- 3) compute C'(x) = C(x) + off_c (signed eval) if needed -----
    have_C = len(terms_c) > 0
    E_c = None
    if have_C:
        E_c = build_eval_from_polynomial(
            n_vars=n_vars, terms=terms_c, acc_bits=c_bits, offset=off_c,
            adder_fn=adder_fn, label="E_c_signed"
        )
        U.compose(E_c, qubits=[*x, *c_acc], inplace=True)

    # ----- 4) flag C >= c_threshold  <=>  C' >= (c_threshold + off_c) -----
    comp_c, ac, c_set_const_true = None, None, False
    c_prime = c_threshold + off_c
    if not have_C:
        # C ≡ 0, so C' ≡ off_c; evaluate trivially
        if c_prime <= 0:
            U.x(c_flag[0])       # always true
            c_set_const_true = True
        # elif c_prime >= 2^c_bits: impossible, cf stays 0
        # else: still false since c_acc is |0...0| and off_c is handled in logic above
    else:
        if c_prime <= 0:
            U.x(c_flag[0])       # always true
            c_set_const_true = True
        elif c_prime >= (1 << c_bits):
            pass                  # impossible: cf stays 0
        else:
            comp_c = IntegerComparator(num_state_qubits=c_bits, value=int(c_prime), geq=True)
            ac_n = comp_c.num_qubits - (c_bits + 1)
            ac = QuantumRegister(ac_n, "ac") if ac_n > 0 else None
            if ac is not None:
                U.add_register(ac)
                U.compose(comp_c, qubits=[*c_acc, c_flag[0], *ac], inplace=True)
            else:
                U.compose(comp_c, qubits=[*c_acc, c_flag[0]], inplace=True)

    # ----- 5) mark: flip y iff ff=cf=1 -----
    U.append(MCXGate(2), qargs=[f_flag[0], c_flag[0], y[0]])

    # ----- 6) uncompute in reverse (including symmetric X if we set const-true) -----
    # Un-flag C
    if comp_c is not None:
        if ac is not None:
            U.compose(comp_c.inverse(), qubits=[*c_acc, c_flag[0], *ac], inplace=True)
        else:
            U.compose(comp_c.inverse(), qubits=[*c_acc, c_flag[0]], inplace=True)
    elif c_set_const_true:
        U.x(c_flag[0])  # undo the X

    # Uncompute C
    if have_C and E_c is not None:
        U.compose(E_c.inverse(), qubits=[*x, *c_acc], inplace=True)

    # Un-flag F
    if comp_f is not None:
        if af is not None:
            U.compose(comp_f.inverse(), qubits=[*f_acc, f_flag[0], *af], inplace=True)
        else:
            U.compose(comp_f.inverse(), qubits=[*f_acc, f_flag[0]], inplace=True)
    elif f_set_const_true:
        U.x(f_flag[0])  # undo the X

    # Uncompute F
    U.compose(E_f.inverse(), qubits=[*x, *f_acc], inplace=True)

    return U.to_gate(label=label)


## Step 3: Iteratively run Grover and update $t$



In [ ]:
from typing import List, Tuple, Optional, Dict, Iterable, Callable
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Gate
from qiskit.quantum_info import Statevector

# ---- Prep & Diffuser (unchanged) ----
def build_prep(n_vars: int) -> Gate:
    x = QuantumRegister(n_vars, "x")
    qc = QuantumCircuit(x, name="Prep")
    qc.h(x)
    return qc.to_gate()

def build_diffuser(n_vars: int, *, Prep: Gate | None = None) -> Gate:
    x = QuantumRegister(n_vars, "x")
    qc = QuantumCircuit(x, name="D")
    if Prep is None:
        Prep = build_prep(n_vars)
    qc.compose(Prep, x, inplace=True)
    qc.x(x)
    if n_vars == 1:
        qc.z(x[0])
    elif n_vars == 2:
        qc.h(x[1]); qc.cz(x[0], x[1]); qc.h(x[1])
    else:
        qc.h(x[-1]); qc.mcx(list(x)[:-1], x[-1]); qc.h(x[-1])
    qc.x(x)
    qc.compose(Prep.inverse(), x, inplace=True)
    return qc.to_gate()

def bbht_schedule(N: int, lam: float = 8/7) -> Iterable[int]:
    m = 1
    while True:
        yield np.random.randint(0, m)  # K in {0,...,m-1}
        m = min(int(np.ceil(lam * m)), int(np.sqrt(N)))

# ---- Robust: build Grover circuit by mirroring the oracle's registers exactly ----
def _build_from_oracle_gate(
    U_gate: Gate,
    K: int,
    *,
    Prep: Gate,
    D: Gate,
    n_vars: int,
    x_reg_name: str = "x",
    label: str = "Grover",
) -> tuple[QuantumCircuit, list]:
    """
    Mirrors *all* qregs from U_gate.definition (names & sizes preserved if present).
    Applies Prep on the x-qubits, then K times (oracle + diffuser on x).
    Works whether the oracle has a named 'x' register or has been flattened to a single 'q' register.

    Returns:
        (qc, x_qubits) where x_qubits is the exact list of top-level qubits used for x (Qubit objects).
    """
    defn = U_gate.definition
    if defn is None:
        raise ValueError("Oracle Gate has no definition; please pass a composite gate.")

    # Mirror every register by name/size (or the single flattened register)
    top_regs = [QuantumRegister(r.size, r.name) for r in defn.qregs]
    qc = QuantumCircuit(*top_regs, name=f"{label}_K={K}")

    name_to_top = {r.name: r for r in qc.qregs}

    # Build mapped_qubits in the exact definition order (this is the qarg order for appending U_gate)
    mapped_qubits = []
    for reg in defn.qregs:
        mapped_qubits.extend(list(name_to_top[reg.name]))

    # Determine which *qubits* are x:
    if x_reg_name in name_to_top:
        x_qubits = list(name_to_top[x_reg_name])
    else:
        # Assume x occupies the first n_vars qubits in the oracle definition
        if n_vars > len(mapped_qubits):
            raise ValueError(f"n_vars={n_vars} exceeds total oracle qubits={len(mapped_qubits)}")
        x_qubits = mapped_qubits[:n_vars]

    # Prep / Diffuser on x (resize if needed)
    if len(x_qubits) != Prep.num_qubits:
        Prep = build_prep(len(x_qubits))
        D = build_diffuser(len(x_qubits), Prep=Prep)
    qc.compose(Prep, qubits=x_qubits, inplace=True)

    # K Grover iterations
    for _ in range(K):
        qc.append(U_gate, qargs=mapped_qubits)
        qc.compose(D, qubits=x_qubits, inplace=True)

    return qc, x_qubits

# ---- Constrained marker-oracle builder (SIGNED) ----
def _build_constrained_marker_gate(
    n_vars: int,
    terms_f: List[Tuple[int, List[int]]],
    t: int,
    terms_c: List[Tuple[int, List[int]]],
    c_threshold: int,
    *,
    adder_fn: Callable[[int, int], QuantumCircuit] = quantum_adder,
) -> Gate:
    # Use SIGNED sizing; the oracle applies the proper shifts internally
    f_bits, _off_f = signed_bits_and_offset_from_terms(terms_f)
    c_bits, _off_c = signed_bits_and_offset_from_terms(terms_c) if terms_c else (1, 0)

    return build_oracle_U_f_t_C(
        n_vars=n_vars,
        terms_f=terms_f,
        t=t,
        terms_c=terms_c,
        c_threshold=c_threshold,
        f_bits=f_bits,
        c_bits=c_bits,
        adder_fn=adder_fn,
        label="U_f>t_and_C>=thr_signed",
    )

def grover_find_one_improving_statevector_constrained(
    n_vars: int,
    terms_f: List[Tuple[int, List[int]]],
    t: int,
    terms_c: List[Tuple[int, List[int]]],
    c_threshold: int,
    max_trials: int = 60,
    lam: float = 8/7,
) -> Optional[Tuple[List[int], int]]:
    """
    BBHT-style randomized Grover with constraint: return one x with
    F(x) > t and C(x) >= c_threshold, else None. Works for SIGNED weights.
    """
    U_gate = _build_constrained_marker_gate(n_vars, terms_f, t, terms_c, c_threshold, adder_fn=quantum_adder)

    Prep = build_prep(n_vars)
    D = build_diffuser(n_vars, Prep=Prep)

    N = 2 ** n_vars
    sched = bbht_schedule(N, lam=lam)

    for _ in range(max_trials):
        K = next(sched)

        # Build Grover circuit and get the exact x-qubits (works even if oracle has only 'q')
        qc, x_qubits = _build_from_oracle_gate(
            U_gate, K, Prep=Prep, D=D, n_vars=n_vars, x_reg_name="x", label="Grover_F>t,C"
        )

        # qargs must be integer indices, not Qubit objects
        x_idx = [qc.find_bit(q).index for q in x_qubits]

        sv = Statevector.from_instruction(qc)
        probs_x = sv.probabilities_dict(qargs=x_idx)  # keys are big-endian over these subsystems
        if not probs_x:
            continue

        keys = list(probs_x.keys())
        vals = np.array([probs_x[k] for k in keys], dtype=float)
        s = vals.sum()
        if s == 0:
            continue
        vals = vals / s

        kidx = np.random.choice(len(keys), p=vals)
        bitstr_x_be = keys[kidx]

        # Return LSB-first per your convention
        x_bits = [int(b) for b in bitstr_x_be][::-1]

        # Evaluate F(x) classically (signed weights OK)
        Fx = sum(w * int(all(x_bits[i] == 1 for i in S)) for (w, S) in terms_f)
        if Fx > t:
            return x_bits, Fx

    return None

# ---- GAS outer loop (constrained, SIGNED): iteratively update t ← F(x) ----
def GAS_maximize_statevector_constrained(
    n_vars: int,
    terms_f: List[Tuple[int, List[int]]],
    terms_c: List[Tuple[int, List[int]]],
    c_threshold: int = 0,
    t_init: int = -1,
    rounds: int = 10,
    trials_per_round: int = 60,
    lam: float = 8/7,
    verbose: bool = True,
) -> Dict[str, object]:
    """
    Grover Adaptive Search with constraint C(x) >= c_threshold (maximize SIGNED F).
    Repeats: find x with F(x) > t and C(x) >= c_threshold; then set t <- F(x).
    Stops when no improvement is found.
    """
    t = t_init
    best_x, best_f = None, None
    history: List[Dict[str, object]] = []

    for r in range(1, rounds + 1):
        if verbose:
            print(f"[Round {r}] threshold t = {t}")

        res = grover_find_one_improving_statevector_constrained(
            n_vars=n_vars,
            terms_f=terms_f,
            t=t,
            terms_c=terms_c,
            c_threshold=c_threshold,
            max_trials=trials_per_round,
            lam=lam,
        )
        if res is None:
            if verbose:
                print("  No improving feasible x found this round. Stopping.")
            break

        x_bits, Fx = res
        history.append({"round": r, "x": x_bits, "F": Fx, "t_before": t, "t_after": Fx})
        t = Fx
        best_x, best_f = x_bits, Fx

        if verbose:
            print(f"  Found x = {x_bits} (LSB-first),  F(x) = {Fx}  → new t = {t}")

    return {"best_x": best_x, "best_f": best_f, "history": history, "rounds_run": len(history)}


# Example:

$f(x)=-2 + 3 x_0 - 4 x_1 x_2 + 5 x_2$, $t=-5$, and $C(x)=x_1+x_3\ge 1$

In [ ]:
# --- Signed objective & constraint ---
n_vars = 3
terms_f = [
    (-2, []),      # constant -2
    ( 3, [0]),     # +3 x0
    (-4, [1,2]),   # -4 x1 x2
    ( 5, [2]),     # +5 x2
]
terms_c = [
    (1, [0]),      # x0
    (1, [1]),      # x1
]
c_threshold = 1   # require x0 + x1 >= 1
t_init = -5

# --- brute-force check to know the ground truth ---
def F_val(x):
    x0,x1,x2 = x
    return -2 + 3*x0 - 4*(x1 & x2) + 5*x2

def C_val(x):
    x0,x1,x2 = x
    return x0 + x1

best = None
best_x = None
for x0 in [0,1]:
    for x1 in [0,1]:
        for x2 in [0,1]:
            x = [x0,x1,x2]
            if C_val(x) >= c_threshold:
                v = F_val(x)
                if (best is None) or (v > best):
                    best, best_x = v, x
print("[Brute] best feasible x, F(x):", best_x, best)

# --- Run signed, constrained GAS ---
res = GAS_maximize_statevector_constrained(
    n_vars=n_vars,
    terms_f=terms_f,
    terms_c=terms_c,
    c_threshold=c_threshold,
    t_init=t_init,
    rounds=5,
    trials_per_round=20,
    verbose=True,
)
print("\n[GAS] result:", res)


[Brute] best feasible x, F(x): [1, 0, 1] 6
[Round 1] threshold t = -5
  Found x = [1, 0, 0] (LSB-first),  F(x) = 1  → new t = 1
[Round 2] threshold t = 1
  Found x = [1, 0, 1] (LSB-first),  F(x) = 6  → new t = 6
[Round 3] threshold t = 6
  No improving feasible x found this round. Stopping.

[GAS] result: {'best_x': [1, 0, 1], 'best_f': 6, 'history': [{'round': 1, 'x': [1, 0, 0], 'F': 1, 't_before': -5, 't_after': 1}, {'round': 2, 'x': [1, 0, 1], 'F': 6, 't_before': 1, 't_after': 6}], 'rounds_run': 2}


# GAS complexity analysis

Let:
- $n$: #variables, $N=2^n$
- $M_f, M_c$: #terms in $f$, $C$
- $d_f^{\max}, d_c^{\max}$: max monomial degree in $f$, $C$
- $m_f, m_c$: bit-widths of $f$, $C$ accumulators (≈ $\log$ of value ranges)

We use:
- **Signed evaluators** via shift $F' = F + \text{offset}$, $C' = C + \text{offset}$  
- **Ancilla-free QFT constant adders** (controlled by monomial literals)  
- **Unsigned IntegerComparator** on shifted values  
- **Standard diffuser** on the $x$-register

---

## One oracle call $U_{f,t,C}$

**Gate count (dominated by controlled constant adders):**
$$
\tilde O\!\big(M_f\, d_f^{\max}\, m_f^2 \;+\; M_c\, d_c^{\max}\, m_c^2 \;+\; m_f \;+\; m_c\big)
$$

**Depth:**
$$
\tilde O\!\big(M_f\, d_f^{\max}\, m_f \;+\; M_c\, d_c^{\max}\, m_c\big)
$$

**Ancillas:**
$$
m_f + m_c + O(1)
$$

---

## One Grover iteration (oracle + diffuser)

**Gate count:**
$$
\tilde O\!\big(M_f d_f^{\max} m_f^2 + M_c d_c^{\max} m_c^2 + n\big)
$$

**Depth:**
$$
\tilde O\!\big(M_f d_f^{\max} m_f + M_c d_c^{\max} m_c + n\big)
$$

**Ancillas:** same as oracle

---

## GAS overall (expected)

Using BBHTscheduling, **expected #oracle calls**:
$$
\tilde O(\sqrt{N}) = \tilde O(2^{n/2})
$$

Multiply per-iteration costs:

**Total gate count:**
$$
\tilde O\!\Big(2^{n/2}\big(M_f d_f^{\max} m_f^2 + M_c d_c^{\max} m_c^2 + n\big)\Big)
$$

**Total depth:**
$$
\tilde O\!\Big(2^{n/2}\big(M_f d_f^{\max} m_f + M_c d_c^{\max} m_c + n\big)\Big)
$$

**Peak ancillas:** $m_f + m_c + O(1)$

---

## Simplifications

If degrees are **constant** and $M_f, M_c = \text{poly}(n)$, $m_f, m_c = O(\log n)$:
  - per-iteration depth $\tilde O(n)$,
  - total depth and total gate $\tilde O(n\,2^{n/2})$,
  - peak ancillas $O(\log n)$.
